In [1]:
import pandas as pd

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [11]:
df = pd.read_csv("preprocessed_electric_vehicle_trip_data.csv")
df.head()

,vehID,step,acceleration(m/s²),actualBatteryCapacity(Wh),SoC(%),speed(m/s),speedFactor,totalEnergyConsumed(Wh),totalEnergyRegenerated(Wh),lon,lat,alt,slope(º),completedDistance(km),mWh,remainingRange(km),time,energyConsumptionRate(Wh/km)
0,EV0,101,0.75,0.292495,1.000000,0.040074,0.202206,0.000036,0.0,0.009050,0.000040,0.924076,0.578218,0.000039,0.065912,0.043997,101,0.937392
1,EV0,102,0.75,0.292448,0.999685,0.080148,0.202206,0.000143,0.0,0.008993,0.000159,0.925038,0.578218,0.000077,0.054530,0.034358,102,1.849586
2,EV0,103,0.75,0.292373,0.999370,0.120221,0.202206,0.000312,0.0,0.008906,0.000337,0.926481,0.578218,0.000174,0.049856,0.030394,103,1.789669
3,EV0,104,0.75,0.292271,0.998740,0.160295,0.202206,0.000543,0.0,0.008791,0.000575,0.928406,0.578218,0.000290,0.047240,0.028170,104,1.868065
4,EV1,104,0.85,0.292478,0.999685,0.061446,0.355191,0.000075,0.0,0.009035,0.000071,0.924332,0.578218,0.000039,0.047814,0.028677,104,1.945341


In [12]:
from sklearn.model_selection import train_test_split

# Selecting relevant features for prediction
features = [
    "acceleration(m/s²)", "speed(m/s)", "speedFactor",
    "totalEnergyConsumed(Wh)", "totalEnergyRegenerated(Wh)",
    "slope(º)", "completedDistance(km)", "remainingRange(km)",
    "energyConsumptionRate(Wh/km)"
]
target = "SoC(%)"

# Splitting data into training (80%) and testing (20%) sets
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (26398, 9)
Testing set size: (6600, 9)


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import numpy as np

# Step 1: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 2: Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Step 3: Make predictions on the test set
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_lr = np.clip(y_pred_lr, 0, 100)  # Ensures SoC is within valid range
# Step 4: Evaluate the model
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

mae = mean_absolute_error(y_test, y_pred_lr)  # Replace y_pred_lr with actual predictions


rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))  # Replace y_pred_lr with actual predictions
# print("Linear Regression RMSE:", rmse)

print("Linear Regression RMSE:", rmse_lr)
print("Linear Regression R² Score:", r2_lr)
print("Linear Regression MAE:", mae)


Linear Regression RMSE: 0.03562770183334124
Linear Regression R² Score: 0.9759790637820672
Linear Regression MAE: 0.026589539412412305


In [18]:
!pip install joblib

In [19]:
#saving model
import joblib

# Save the model
joblib.dump(lr_model, "soc_linear_reg_prediction_model.pkl")

# Save the scaler (for consistent preprocessing)
joblib.dump(scaler, "scaler_linear_reg.pkl")


['scaler_linear_reg.pkl']

In [20]:
#load model for predictions
# Load the trained model
lr_model = joblib.load("soc_linear_reg_prediction_model.pkl")
scaler = joblib.load("scaler_linear_reg.pkl")

# Sample new input data (replace with real sensor/API data)
new_data = [[0.75, 12.5, 0.9, 10, 1, 2.5, 1.2, 15, 120]]  # Example input values

# Apply the same scaling transformation
new_data_scaled = scaler.transform(new_data)

# Predict SoC
predicted_soc = lr_model.predict(new_data_scaled)
print("Predicted SoC:", predicted_soc[0])


Predicted SoC: -1.042075988721428


C:\Users\omkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
